In [4]:
import pandas as pd
import numpy as np
# from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
import scipy.stats as stats
from scipy.stats import norm
from scipy.optimize import minimize

# Infant Birthweight

We attempt to predict find the effect of smoking on infant birthweight given censored, and truncated datasets. We use maximum likelihood estimation to generate viable estimations, and compare these to benchmarks trained on a complete dataset. 

In [5]:
df = pd.read_csv(r"C:\Users\jonat\OneDrive\econometrics\microeconometrics\data\paeco526_trun.csv")
df.head()

,alcohol,anemia,cardiac,chyper,dbirwt,dfage,dfeduc,diabete,disllb,dlivord,...,fhispan,adequac2,adequac3,tripre2,tripre3,tripre0,first,deadkids,plural,dmage2
0,0,0,0,0,3487,32,15,0,20,2,...,0,1,0,1,0,0,0,1,0,784
1,0,0,0,0,3033,31,16,0,0,1,...,0,0,0,0,0,0,1,0,0,1024
2,0,0,0,0,3232,24,11,0,13,4,...,0,0,1,0,0,0,0,1,0,729
3,0,0,0,0,5131,23,13,0,0,1,...,0,0,0,0,0,0,1,0,0,484
4,0,0,0,0,3572,31,14,0,0,1,...,0,0,0,0,0,0,1,0,0,900


We begin by training a naive regression model on truncated data (birthweights below 2500 grams are not recorded) to estimate the coefficient for smoking on infant birthweight. This estimate will be biased

In [6]:
# defining the covariates with an intercept

X = df[['tobacco', "dmage", "dmage2", "dmeduc", "mblack", "motherr", "mhispan", "dmar", "foreignb", "dfage",
         "dfeduc", "fblack", "fotherr", "fhispan", "alcohol", "drink", "adequac2", "adequac3", 
         "tripre0", "tripre2", "tripre3", "nprevist", "first", "dlivord", "disllb", "pre4000", 
         "plural", "diabete", "anemia", "cardiac", "chyper"]]
X = sm.add_constant(X)

# defining the dependent variable
y = df["dbirwt"]

# train the model
trun_model = sm.OLS(y, X).fit()
print(trun_model.summary())

# return the coefficient on smoking
smoking_coef = trun_model.params["tobacco"]
print(f"Coefficient on smoking: {smoking_coef}")


                            OLS Regression Results                            
Dep. Variable:                 dbirwt   R-squared:                       0.098
Model:                            OLS   Adj. R-squared:                  0.097
Method:                 Least Squares   F-statistic:                     820.6
Date:                Thu, 13 Feb 2025   Prob (F-statistic):               0.00
Time:                        17:48:43   Log-Likelihood:            -1.7660e+06
No. Observations:              235263   AIC:                         3.532e+06
Df Residuals:                  235231   BIC:                         3.532e+06
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       3159.0308     21.731    145.368      0.0

The coefficient on smoking is roughly -191, which would imply that smoking is associated with babies being born 191 grams lighter. We would expect this estimate to be biased for the same reasons mentioned above; we are removing data from babies with very low birthweights, which will cause the model to miss the association of tobacco in extreme cases. For these resons, we would expect the effect from smoking to be more severe than that given by the model above.

# Deriving the Log Likelihood

Due to the bias inherent in OLS models used on truncated data, we will turn to maximum likelihood to estimate our parameters. 

We begin by assuming that birthweight is given by the latent variable:

$$
y_i^* = x_i'\beta + \epsilon
$$

where $\epsilon$ follows a normal distribution with mean zero and variance $\sigma^2$:

$$
y_i^* \sim N(x_i'\beta, \sigma^2)
$$

However, in practice, we observe $y_i$, which is related to the latent variable $y_i^*$ through truncation:

$$
y_i =
\begin{cases} 
y_i^*, & \text{if } y_i^* > 2500 \\
\text{unobserved}, & \text{otherwise}
\end{cases}
$$

## Likelihood Function

For observed (untruncated) data, the likelihood contribution of an individual observation is given by the probability density function of a normal variable:

$$
f(y_i | x_i, \beta, \sigma) = \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left(-\frac{(y_i - x_i'\beta)^2}{2\sigma^2} \right)
$$

For truncated observations, we must account for the fact that the data is only observed when $y_i^* > 2500$. This means that the probability of observing $y_i$ is conditional on this truncation:

$$
P(y_i^* > 2500) = 1 - \Phi \left(\frac{2500 - x_i'\beta}{\sigma} \right)
$$

Note that here it isn't correct to simply write $P(y_i^* > 2500) = 1 - \Phi(2500)$, as this doesn't account for the truncation of the distribution.

where $\Phi$ is the cumulative distribution function (CDF) of the standard normal distribution.

Thus, the likelihood function for the full dataset is:

$$
L(\beta, \sigma) = \prod_{i \in \text{observed}} \frac{1}{\sqrt{2\pi\sigma^2}} \exp \left(-\frac{(y_i - x_i'\beta)^2}{2\sigma^2} \right) \times \frac{1}{1 - \Phi \left(\frac{2500 - x_i'\beta}{\sigma} \right)}
$$

## Log Likelihood Function

Taking the natural logarithm, we obtain the log likelihood function:

$$
\log L(\beta, \sigma) = \sum_{i \in \text{observed}} \left[ -\frac{1}{2} \log(2\pi\sigma^2) - \frac{(y_i - x_i'\beta)^2}{2\sigma^2} - \log \left( 1 - \Phi \left(\frac{2500 - x_i'\beta}{\sigma} \right) \right) \right]
$$

This function can then be maximized with respect to $\beta$ and $\sigma$ to obtain the maximum likelihood estimates.



# Maximum Likelihood Estimation

For an unbiased estimate, we turn to MLE. We first define the negative log likelihood function, then use numerical approximation methods to find the minimum value, deriving the most likely distribution parameters given the data

In [7]:
# Define the log-likelihood function
def truncated_normal_log_likelihood(params, X, y):
    beta = params[:-1]  # Regression coefficients
    sigma = params[-1]  # Standard deviation
    
    if sigma <= 0:  # Avoid invalid sigma values
        return np.inf
    
    mu = X @ beta  # Mean prediction
    phi = stats.norm.pdf((y - mu) / sigma)  # Normal PDF
    Phi_c = 1 - stats.norm.cdf((2500 - mu) / sigma)  # Survival function (1 - CDF)

    # Log-likelihood
    log_likelihood = np.sum(np.log(phi / (sigma * Phi_c)))  
    return -log_likelihood  # Negative log-likelihood for minimization

In [12]:
# Initial parameter guesses
beta_init = list(trun_model.params)  # Start with regression coefficients for beta
sigma_init = 100  # Initial sigma guess
params_init = np.append(beta_init, sigma_init)

# Optimize MLE
result = minimize(truncated_normal_log_likelihood, params_init, args=(X, y), method="L-BFGS-B")

# Extract estimated parameters
beta_hat = result.x[:-1]
sigma_hat = result.x[-1]

# Compute Hessian for standard errors
hessian_inv = result.hess_inv.todense()  # Inverse Hessian gives variance estimates
standard_errors = np.sqrt(np.diag(hessian_inv))

# 95% confidence intervals
z_critical = stats.norm.ppf(0.975)
ci_lower = beta_hat - z_critical * standard_errors[:-1]
ci_upper = beta_hat + z_critical * standard_errors[:-1]

# Print results
print("Estimated Coefficients and 95% Confidence Intervals:")
print(f"β_1: {beta_hat[1]:.3f} (95% CI: [{ci_lower[1]:.3f}, {ci_upper[1]:.3f}])")

print(f"Estimated σ: {sigma_hat:.3f}")

Estimated Coefficients and 95% Confidence Intervals:
β_1: -193.562 (95% CI: [-222.980, -164.143])
Estimated σ: 470.502


# Censored Birthweight

Now suppose that instead of the birthweight being truncated, the value 'low birth weight' is inputed rather than a numerical value for the weight of the baby. We will follow the same steps as above in estimating the coefficients for censored data, rather than truncated

We begin by estimating a rudimentary OLS model on the data, regessing censored brithweight on the 32 covariates

In [13]:
cens_df = pd.read_csv(r"C:\Users\jonat\OneDrive\econometrics\microeconometrics\data\paeco526_cen_full.csv")

X = cens_df[['tobacco', "dmage", "dmage2", "dmeduc", "mblack", "motherr", "mhispan", "dmar", "foreignb", "dfage",
         "dfeduc", "fblack", "fotherr", "fhispan", "alcohol", "drink", "adequac2", "adequac3", 
         "tripre0", "tripre2", "tripre3", "nprevist", "first", "dlivord", "disllb", "pre4000", 
         "plural", "diabete", "anemia", "cardiac", "chyper"]]
X = sm.add_constant(X)

# defining the dependent variable
y = cens_df["cdbirwt2500"]

# train the model
cens_model = sm.OLS(y, X).fit()
print(cens_model.summary())

# return the coefficient on smoking
smoking_coef = cens_model.params["tobacco"]
print(f"Coefficient on smoking: {smoking_coef}")

                            OLS Regression Results                            
Dep. Variable:            cdbirwt2500   R-squared:                       0.137
Model:                            OLS   Adj. R-squared:                  0.137
Method:                 Least Squares   F-statistic:                     1284.
Date:                Thu, 13 Feb 2025   Prob (F-statistic):               0.00
Time:                        17:59:15   Log-Likelihood:            -1.8918e+06
No. Observations:              250000   AIC:                         3.784e+06
Df Residuals:                  249968   BIC:                         3.784e+06
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2994.0627     22.145    135.203      0.0

As we can see, the estimated coefficient for smoking on birthweight is -213.74. Again, we would not expect this to be an unbiased estimate of the true effect, since the data is a biased representation of the effect of smoking on birthweight. Babies that weighed less than 2500 grams were recorded as weighing 2500 grams. 

Again, we utilize MLE...

# Censored Log Likelihood Function

Assuming the error term $\epsilon$ is i.i.d, $\epsilon \sim N(0, \sigma^2)$, and is independent of the regressors, we derive the log-likelihood using the same steps as above.

Note we are observing a $y_i$, instead of the latent variable $y_i^*$ which actually represents birthweight. We can leverage the assumed distribution of $\epsilon$ to prove the following distribution of $y_i^*$:

$y_i^* = x_i' \beta + \epsilon \implies y_i^* \sim N(x_i'\beta, \sigma^2)$

There are two cases to consider when creating our likelihood functions; the data can be uncensored, or left censored, i.e., the birthweight is below 2500 but recorded as 2500.

1. Uncensored Observations

    In the case where the data is uncensored, we observe the latent variable $y_i^*$, so our likelihood function is simply the pdf of $y_i^*$:

    $f(y_i | x_i, \beta, \sigma) = \frac{1}{\sqrt{2\pi\sigma}}exp(-\frac{(y_i - x_i'\beta)^2}{2\sigma^2})$

2. Censored observations

    If the true birthweight is below 2500, we observe $y_i = 2500$, meaning:

    $P(y_i = 2500 | x_i, \beta, \sigma) = P(y_i^* \leq 2500 | x_i)$

    We can, again, leverage the the fact that $y_i^*$ is normall distributed to rewrite the probability as a cdf of the normal distribution

    $P(y_i^* \leq 2500 | x_i) = \phi(\frac{2500 - x_i'\beta}{\sigma})$

    where $\phi$ is the normal cdf

The full likelihood function is obtained by multiplying the probabilities of all observations:


$L(\beta, \sigma) = \prod_{i \in \mathcal{U}} f(y_i | x_i, \beta, \sigma) \prod_{i \in \mathcal{L}} P(y_i^* \leq 2500 | x_i, \beta, \sigma)$


Taking the log of this function gives us the log-likelihood:


$\log L(\beta, \sigma) = \sum_{i \in \mathcal{U}} \log \frac{1}{\sqrt{2\pi} \sigma} \exp -\frac{(y_i - x_i' \beta)^2}{2\sigma^2} + \sum_{i \in \mathcal{L}} \log \left( \Phi \left( \frac{2500 - x_i' \beta}{\sigma} \right) \right)$

where:

$\mathcal{U}$ represents uncensored

$\mathcal{L}$ represents left-censored

In [14]:
def censored_nll(params, X, y, censored_threshold=2500, censor_mask=None):
    """
    Compute the negative log-likelihood for a censored normal model.

    Parameters:
    - params: array-like, contains [beta_0, beta_1, ..., beta_p, sigma]
    - X: (n, p) array of regressors
    - y: (n, ) array of observed values
    - censored_threshold: float, threshold below which y is censored
    - censor_mask: (n, ) boolean array, True if observation is censored

    Returns:
    - Negative log-likelihood
    """
    # Extract beta coefficients and sigma
    beta = params[:-1]
    sigma = params[-1]
    
    # Ensure sigma is positive
    if sigma <= 0:
        return np.inf

    # Compute predicted latent values
    y_star_mean = X @ beta  # Xβ

    # Compute likelihoods
    uncensored_likelihood = norm.pdf(y, loc=y_star_mean, scale=sigma)
    censored_likelihood = norm.cdf((censored_threshold - y_star_mean) / sigma)

    # Use censor_mask to select appropriate likelihoods
    likelihood = np.where(censor_mask, censored_likelihood, uncensored_likelihood)

    # Avoid log(0) issues by replacing 0s with small positive values
    likelihood = np.maximum(likelihood, 1e-10)

    # Compute negative log-likelihood
    neg_log_lik = -np.sum(np.log(likelihood))

    return neg_log_lik


In [17]:

# Initial parameter guesses
beta_init = list(cens_model.params)  # Start with regression coefficients for beta
sigma_init = 100  # Initial sigma guess
params_init = np.append(beta_init, sigma_init)

# Optimize MLE
cens_result = minimize(censored_nll, params_init, args=(X, y), method="L-BFGS-B")

# Extract estimated parameters
cens_beta_hat = cens_result.x[:-1]
cens_sigma_hat = cens_result.x[-1]

# Compute Hessian for standard errors
hessian_inv = cens_result.hess_inv.todense()  # Inverse Hessian gives variance estimates
standard_errors = np.sqrt(np.diag(hessian_inv))

# 95% confidence intervals
z_critical = stats.norm.ppf(0.975)
ci_lower = cens_beta_hat - z_critical * standard_errors[:-1]
ci_upper = cens_beta_hat + z_critical * standard_errors[:-1]

# Print results
print("Estimated Coefficients and 95% Confidence Intervals:")
print(f"β_1: {cens_beta_hat[1]:.3f} (95% CI: [{ci_lower[1]:.3f}, {ci_upper[1]:.3f}])")

print(f"Estimated σ: {cens_sigma_hat:.3f}")

c:\Users\jonat\OneDrive\econometrics\dynamic_stochastic\.venv\lib\site-packages\scipy\optimize\_numdiff.py:596: RuntimeWarning: invalid value encountered in subtract
  df = fun(x1) - f0


Estimated Coefficients and 95% Confidence Intervals:
β_1: -214.595 (95% CI: [-216.644, -212.546])
Estimated σ: 440.816


# Comparing Results to Regression on Uncensored Data

As a benchmark to test the accuracy of our truncated, and censored models, we run regression the uncensored dataset, and compare the three sets of parameters in a table below:

In [18]:
# defining the dependent variable
y = cens_df["dbirwt"]

# train the model
true_model = sm.OLS(y, X).fit()
print(true_model.summary())

# return the coefficient on smoking
smoking_coef = true_model.params["tobacco"]
print(f"Coefficient on smoking: {smoking_coef}")

                            OLS Regression Results                            
Dep. Variable:                 dbirwt   R-squared:                       0.155
Model:                            OLS   Adj. R-squared:                  0.155
Method:                 Least Squares   F-statistic:                     1478.
Date:                Thu, 13 Feb 2025   Prob (F-statistic):               0.00
Time:                        18:01:52   Log-Likelihood:            -1.9257e+06
No. Observations:              250000   AIC:                         3.851e+06
Df Residuals:                  249968   BIC:                         3.852e+06
Df Model:                          31                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const       2805.6151     25.356    110.647      0.0

In [19]:
# Example coefficients and standard errors for the smoking variable
models = ["trunc_ols", "trunc_mle", "cens_ols", "cens_mle", "true_ols"]
coefficients = [trun_model.params['tobacco'], 
                beta_hat[1], 
                cens_model.params['tobacco'], 
                cens_beta_hat[1], 
                true_model.params['tobacco']]  # Example values

# Create a DataFrame
results_df = pd.DataFrame({
    "Model": models,
    "Smoking Coefficient": coefficients,
})

# Display as a table in Jupyter Notebook
from IPython.display import display
display(results_df)

,Model,Smoking Coefficient
0,trunc_ols,-190.993976
1,trunc_mle,-193.561516
2,cens_ols,-213.735631
3,cens_mle,-214.594893
4,true_ols,-229.418522
